In [6]:
import pandas as pd
import nasdaqdatalink as ndl
import json
import numpy as np
from typing import List, Callable, Dict
from realgam.quantlib import general_utils as gu
from realgam.quantlib.engineer.op_engineer_vect import OpEngineerV
from realgam.quantlib.engineer.ta_engineer_vect import TalibEngineerV
from realgam.quantlib.engineer.alpha_engineer_vect import AlphaEngineerV
from realgam.quantlib.engineer.test_no_class import alpha001, ts_ret
import talib
from joblib import Parallel, delayed
import datetime
import os
import time
PROJECT_PATH = os.getenv('QuantSystemMVP')
DATA_PATH = f'{PROJECT_PATH}/Data/stock_hist_perma.obj'
ENG_PATH = f'{PROJECT_PATH}/Data/test_alpha_2022-03-28.obj'
NDL_CONFIG_PATH = f'{PROJECT_PATH}/realgam/quantlib/nasdaq_dl_config.json'
NDL_CONFIG = json.load(open(NDL_CONFIG_PATH))

In [8]:
eng = gu.load_file(ENG_PATH)
oe = OpEngineerV(eng)

In [9]:
oe.ts_lag('volume', -6)

permaticker  date      
101501       2012-01-03         0.0
104304       2012-01-03         0.0
104668       2012-01-03       460.0
104825       2012-01-03         0.0
105088       2012-01-03         0.0
                             ...   
638355       2022-03-18    230311.0
638356       2022-03-18     47976.0
638366       2022-03-18    242723.0
638371       2022-03-18     44602.0
638372       2022-03-18    551049.0
Length: 11981064, dtype: float64

In [10]:
oe.ts_retn('closeadj', 6, inplace=True)
eng['ret6_fwd'] = oe.ts_lag('ts_retn6_closeadj', -6)

In [ ]:
eng['ret6'] = eng.ret6.shift(-6)

In [18]:
tae = TalibEngineerV(eng)
eng['adv20'] = tae.adv(20)

In [22]:
oe.set_df(eng)
eng['cs_rank_adv20'] = oe.cs_rank('adv20')

In [25]:
eng[eng.cs_rank_adv20 > 2500]

ticker   open   high     low  close  openadj  highadj  \
permaticker date                                                               
104612      2022-01-10   APCX   2.25   2.25   1.710   1.89     2.25     2.25   
            2022-01-11   APCX   1.89   2.18   1.791   2.15     1.89     2.18   
            2022-01-12   APCX   2.05   2.39   1.990   2.24     2.05     2.39   
            2022-01-13   APCX   2.17   2.23   2.030   2.08     2.17     2.23   
            2022-01-14   APCX   2.05   2.41   2.050   2.22     2.05     2.41   
...                       ...    ...    ...     ...    ...      ...      ...   
638186      2022-03-23  CLINU  10.02  10.02  10.020  10.02    10.02    10.02   
638201      2022-03-24  GSRMU  10.25  10.25  10.080  10.08    10.25    10.25   
            2022-03-25  GSRMU  10.23  10.46  10.070  10.10    10.23    10.46   
638202      2022-03-24  FGMCU  10.03  10.03  10.030  10.03    10.03    10.03   
638234      2022-03-28   ZIMV  22.95  23.37  22.340  22.69    22.95    23.37   

                        lowadj  closeadj    volume    alpha1    alpha2  \
permaticker date                                                         
104612      2022-01-10   1.710      1.89  563129.0 -0.236569 -0.371428   
            2022-01-11   1.791      2.15  135640.0 -0.111537  0.306276   
            2022-01-12   1.990      2.24  121699.0 -0.058054  0.500763   
            2022-01-13   2.030      2.08   64704.0 -0.346554  0.367854   
            2022-01-14   2.050      2.22  136409.0 -0.277474 -0.389261   
...                        ...       ...       ...       ...       ...   
638186      2022-03-23  10.020     10.02  650020.0       NaN -0.376691   
638201      2022-03-24  10.080     10.08   15559.0       NaN  0.078507   
            2022-03-25  10.070     10.10   21842.0       NaN  0.322677   
638202      2022-03-24  10.030     10.03       0.0       NaN  0.436663   
638234      2022-03-28  22.340     22.69  201997.0       NaN  0.212196   

                          alpha3  alpha4    alpha5    alpha6  alpha7  \
permaticker date                                                       
104612      2022-01-10 -0.963897    -5.0 -0.119659 -0.896131   -55.0   
            2022-01-11 -0.959182    -5.0 -0.523857 -0.884942    -1.0   
            2022-01-12 -0.964285    -6.0 -0.640489 -0.859963    -1.0   
            2022-01-13 -0.936529    -6.0 -0.340535 -0.819221    -1.0   
            2022-01-14 -0.930440    -6.0 -0.312737 -0.843895    -1.0   
...                          ...     ...       ...       ...     ...   
638186      2022-03-23 -0.200541    -6.0 -0.244275 -0.583565     NaN   
638201      2022-03-24  0.407207    -5.0 -0.069187  0.505003     NaN   
            2022-03-25  0.458525    -1.0 -0.110239  0.624941     NaN   
638202      2022-03-24  0.116799    -2.0 -0.138332  0.711221     NaN   
638234      2022-03-28  0.104760    -1.0 -0.010337  0.500824     NaN   

                        ret6_fwd       adv20  cs_rank_adv20  
permaticker date                                             
104612      2022-01-10  0.158730   184505.90         2575.0  
            2022-01-11 -0.072093   191152.90         2625.0  
            2022-01-12 -0.138393   196837.85         2662.0  
            2022-01-13 -0.134615   199983.05         2713.0  
            2022-01-14 -0.238739   206573.50         2763.0  
...                          ...         ...            ...  
638186      2022-03-23       NaN  1256371.60         4592.0  
638201      2022-03-24       NaN   941733.80         4295.0  
            2022-03-25       NaN   225554.95         2586.0  
638202      2022-03-24       NaN   165475.90         2528.0  
638234      2022-03-28       NaN  1078956.30         4137.0  

[5446210 rows x 20 columns]

ticker    open    high    low   close  openadj  \
permaticker date                                                         
101501      2012-01-03   BBUCQ   0.030   0.030   0.03   0.030    0.030   
            2012-01-04   BBUCQ   0.030   0.030   0.03   0.030    0.030   
            2012-01-05   BBUCQ   0.030   0.030   0.03   0.030    0.030   
            2012-01-06   BBUCQ   0.030   0.030   0.03   0.030    0.030   
            2012-01-09   BBUCQ   0.030   0.030   0.03   0.030    0.030   
...                        ...     ...     ...    ...     ...      ...   
638366      2022-03-25    ANTX  16.850  17.800  15.00  15.400   16.850   
            2022-03-28    ANTX  15.010  15.770  15.00  15.380   15.010   
638368      2022-03-24  AXAC.U  10.180  10.180  10.18  10.180   10.180   
638371      2022-03-28   MURFU  10.375  10.375  10.07  10.100   10.375   
638372      2022-03-28   TGR.U  10.070  10.110  10.07  10.092   10.070   

                        highadj  lowadj  closeadj     volume  alpha1  alpha2  \
permaticker date                                                               
101501      2012-01-03    0.030    0.03     0.030        0.0     NaN     NaN   
            2012-01-04    0.030    0.03     0.030        0.0     NaN     NaN   
            2012-01-05    0.030    0.03     0.030        0.0     NaN     NaN   
            2012-01-06    0.030    0.03     0.030        0.0     NaN     NaN   
            2012-01-09    0.030    0.03     0.030        0.0     NaN     NaN   
...                         ...     ...       ...        ...     ...     ...   
638366      2022-03-25   17.800   15.00    15.400  1389068.0     NaN     NaN   
            2022-03-28   15.770   15.00    15.380   242723.0     NaN     NaN   
638368      2022-03-24   10.180   10.18    10.180   100000.0     NaN     NaN   
638371      2022-03-28   10.375   10.07    10.100    44602.0     NaN     NaN   
638372      2022-03-28   10.110   10.07    10.092   551049.0     NaN     NaN   

                        alpha3  alpha4  alpha5  alpha6  alpha7  ret6_fwd  \
permaticker date                                                           
101501      2012-01-03     NaN     NaN     NaN     NaN     NaN       0.0   
            2012-01-04     NaN     NaN     NaN     NaN     NaN       0.0   
            2012-01-05     NaN     NaN     NaN     NaN     NaN       0.0   
            2012-01-06     NaN     NaN     NaN     NaN     NaN       0.0   
            2012-01-09     NaN     NaN     NaN     NaN     NaN       0.0   
...                        ...     ...     ...     ...     ...       ...   
638366      2022-03-25     NaN     NaN     NaN     NaN     NaN       NaN   
            2022-03-28     NaN     NaN     NaN     NaN     NaN       NaN   
638368      2022-03-24     NaN     NaN     NaN     NaN     NaN       NaN   
638371      2022-03-28     NaN     NaN     NaN     NaN     NaN       NaN   
638372      2022-03-28     NaN     NaN     NaN     NaN     NaN       NaN   

                        adv20  cs_rank_adv20  
permaticker date                              
101501      2012-01-03    NaN            NaN  
            2012-01-04    NaN            NaN  
            2012-01-05    NaN            NaN  
            2012-01-06    NaN            NaN  
            2012-01-09    NaN            NaN  
...                       ...            ...  
638366      2022-03-25    NaN            NaN  
            2022-03-28    NaN            NaN  
638368      2022-03-24    NaN            NaN  
638371      2022-03-28    NaN            NaN  
638372      2022-03-28    NaN            NaN  

[12009230 rows x 20 columns]

In [26]:
eng_pre = eng.dropna()
eng_pre = eng_pre[eng_pre.cs_rank_adv20 > 2500]

In [32]:
eng_pre['ret6_fwd'] = np.where(eng_pre.ret6_fwd > 0.001, 1, 0)

In [43]:
gu.save_file(f'{PROJECT_PATH}/Data/feature/alpha_raw_20220330.obj', eng)
gu.save_file(f'{PROJECT_PATH}/Data/feature/alpha_pre_20220330.obj', eng_pre)

In [44]:
eng_pre

ticker  open  high    low  close  openadj  highadj  \
permaticker date                                                            
104612      2022-01-10   APCX  2.25  2.25  1.710   1.89     2.25     2.25   
            2022-01-11   APCX  1.89  2.18  1.791   2.15     1.89     2.18   
            2022-01-12   APCX  2.05  2.39  1.990   2.24     2.05     2.39   
            2022-01-13   APCX  2.17  2.23  2.030   2.08     2.17     2.23   
            2022-01-14   APCX  2.05  2.41  2.050   2.22     2.05     2.41   
...                       ...   ...   ...    ...    ...      ...      ...   
637794      2022-03-14   SBFM  2.81  3.05  2.690   2.89     2.81     3.05   
            2022-03-15   SBFM  2.81  4.75  2.772   3.27     2.81     4.75   
            2022-03-16   SBFM  3.06  3.51  2.870   3.28     3.06     3.51   
            2022-03-17   SBFM  3.06  3.25  2.970   3.03     3.06     3.25   
            2022-03-18   SBFM  3.46  3.48  3.100   3.25     3.46     3.48   

                        lowadj  closeadj     volume    alpha1    alpha2  \
permaticker date                                                          
104612      2022-01-10   1.710      1.89   563129.0 -0.236569 -0.371428   
            2022-01-11   1.791      2.15   135640.0 -0.111537  0.306276   
            2022-01-12   1.990      2.24   121699.0 -0.058054  0.500763   
            2022-01-13   2.030      2.08    64704.0 -0.346554  0.367854   
            2022-01-14   2.050      2.22   136409.0 -0.277474 -0.389261   
...                        ...       ...        ...       ...       ...   
637794      2022-03-14   2.690      2.89   490633.0 -0.166695  0.637373   
            2022-03-15   2.772      3.27  7306862.0 -0.102859  0.430296   
            2022-03-16   2.870      3.28  1090079.0 -0.368640  0.371101   
            2022-03-17   2.970      3.03   867434.0 -0.171894 -0.426732   
            2022-03-18   3.100      3.25   840772.0 -0.111319 -0.419262   

                          alpha3  alpha4    alpha5    alpha6  alpha7  \
permaticker date                                                       
104612      2022-01-10 -0.963897    -5.0 -0.119659 -0.896131   -55.0   
            2022-01-11 -0.959182    -5.0 -0.523857 -0.884942    -1.0   
            2022-01-12 -0.964285    -6.0 -0.640489 -0.859963    -1.0   
            2022-01-13 -0.936529    -6.0 -0.340535 -0.819221    -1.0   
            2022-01-14 -0.930440    -6.0 -0.312737 -0.843895    -1.0   
...                          ...     ...       ...       ...     ...   
637794      2022-03-14 -0.922962    -7.0 -0.777463 -0.818239    -1.0   
            2022-03-15 -0.919751    -7.0 -0.081093 -0.804018   -35.0   
            2022-03-16 -0.904994    -7.0 -0.493559 -0.742509    -1.0   
            2022-03-17 -0.879794    -6.0 -0.054808 -0.702113    -1.0   
            2022-03-18 -0.775873    -7.0 -0.093871 -0.627809    -1.0   

                        ret6_fwd       adv20  cs_rank_adv20  
permaticker date                                             
104612      2022-01-10         1   184505.90         2575.0  
            2022-01-11         0   191152.90         2625.0  
            2022-01-12         0   196837.85         2662.0  
            2022-01-13         0   199983.05         2713.0  
            2022-01-14         0   206573.50         2763.0  
...                          ...         ...            ...  
637794      2022-03-14         0  3890000.10         5470.0  
            2022-03-15         0  4254923.55         5507.0  
            2022-03-16         0  4176195.25         5494.0  
            2022-03-17         0  4162609.00         5483.0  
            2022-03-18         0  4158389.85         5462.0  

[5267783 rows x 20 columns]

In [ ]:
# def cvScore(clf,X,y,sample_weight,scoring='neg_log_loss',t1=None,cv=None,cvGen=None,pctEmbargo=None):
#     if scoring not in ['neg_log_loss','accuracy']:
#         raise Exception('wrong scoring method.')
#     from sklearn.metrics import log_loss,accuracy_score, f1_score, precision_score, recall_score
#     # from clfSequential import PurgedKFold
#     from sklearn.model_selection import KFold
#     if cvGen is None:
#         cvGen= KFold(n_splits=cv) # purged
#     score=[]
#     for i, (train,test) in enumerate(cvGen.split(X=X)):
#         fit=clf.fit(X=X.iloc[train,:],y=y.iloc[train],sample_weight=sample_weight.iloc[train].values)
#         if scoring=='neg_log_loss':
#             prob=fit.predict_proba(X.iloc[test,:])
#             score_=-log_loss(y.iloc[test],prob,sample_weight=sample_weight.iloc[test].values,labels=clf.classes_)
#         else:
#             pred=fit.predict(X.iloc[test,:])
#             score_=accuracy_score(y.iloc[test],pred,sample_weight= sample_weight.iloc[test].values)
#         score.append(score_)
#     return np.array(score)

def cvScore(clf,X,y,sample_weight = 1,scoring='neg_log_loss',t1=None,cv=None,cvGen=None,pctEmbargo=None):
    if scoring not in ['neg_log_loss','accuracy']:
        raise Exception('wrong scoring method.')
    from sklearn.metrics import log_loss,accuracy_score, f1_score, precision_score, recall_score
    # from clfSequential import PurgedKFold
    from sklearn.model_selection import KFold
    if cvGen is None:
        cvGen= KFold(n_splits=cv) # purged
    score=[]
    for i, (train,test) in enumerate(cvGen.split(X=X)):
        fit=clf.fit(X=X.iloc[train,:],y=y.iloc[train],sample_weight=sample_weight.iloc[train].values)
        if scoring=='neg_log_loss':
            prob=fit.predict_proba(X.iloc[test,:])
            score_=-log_loss(y.iloc[test],prob,sample_weight=sample_weight.iloc[test].values,labels=clf.classes_)
        else:
            pred=fit.predict(X.iloc[test,:])
            score_=accuracy_score(y.iloc[test],pred,sample_weight= sample_weight.iloc[test].values)
        score.append(score_)
    return np.array(score)

In [31]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier


In [57]:
eng_pre

ticker  open  high    low  close  openadj  highadj  \
permaticker date                                                            
104612      2022-01-10   APCX  2.25  2.25  1.710   1.89     2.25     2.25   
            2022-01-11   APCX  1.89  2.18  1.791   2.15     1.89     2.18   
            2022-01-12   APCX  2.05  2.39  1.990   2.24     2.05     2.39   
            2022-01-13   APCX  2.17  2.23  2.030   2.08     2.17     2.23   
            2022-01-14   APCX  2.05  2.41  2.050   2.22     2.05     2.41   
...                       ...   ...   ...    ...    ...      ...      ...   
637794      2022-03-14   SBFM  2.81  3.05  2.690   2.89     2.81     3.05   
            2022-03-15   SBFM  2.81  4.75  2.772   3.27     2.81     4.75   
            2022-03-16   SBFM  3.06  3.51  2.870   3.28     3.06     3.51   
            2022-03-17   SBFM  3.06  3.25  2.970   3.03     3.06     3.25   
            2022-03-18   SBFM  3.46  3.48  3.100   3.25     3.46     3.48   

                        lowadj  closeadj     volume    alpha1    alpha2  \
permaticker date                                                          
104612      2022-01-10   1.710      1.89   563129.0 -0.236569 -0.371428   
            2022-01-11   1.791      2.15   135640.0 -0.111537  0.306276   
            2022-01-12   1.990      2.24   121699.0 -0.058054  0.500763   
            2022-01-13   2.030      2.08    64704.0 -0.346554  0.367854   
            2022-01-14   2.050      2.22   136409.0 -0.277474 -0.389261   
...                        ...       ...        ...       ...       ...   
637794      2022-03-14   2.690      2.89   490633.0 -0.166695  0.637373   
            2022-03-15   2.772      3.27  7306862.0 -0.102859  0.430296   
            2022-03-16   2.870      3.28  1090079.0 -0.368640  0.371101   
            2022-03-17   2.970      3.03   867434.0 -0.171894 -0.426732   
            2022-03-18   3.100      3.25   840772.0 -0.111319 -0.419262   

                          alpha3  alpha4    alpha5    alpha6  alpha7  \
permaticker date                                                       
104612      2022-01-10 -0.963897    -5.0 -0.119659 -0.896131   -55.0   
            2022-01-11 -0.959182    -5.0 -0.523857 -0.884942    -1.0   
            2022-01-12 -0.964285    -6.0 -0.640489 -0.859963    -1.0   
            2022-01-13 -0.936529    -6.0 -0.340535 -0.819221    -1.0   
            2022-01-14 -0.930440    -6.0 -0.312737 -0.843895    -1.0   
...                          ...     ...       ...       ...     ...   
637794      2022-03-14 -0.922962    -7.0 -0.777463 -0.818239    -1.0   
            2022-03-15 -0.919751    -7.0 -0.081093 -0.804018   -35.0   
            2022-03-16 -0.904994    -7.0 -0.493559 -0.742509    -1.0   
            2022-03-17 -0.879794    -6.0 -0.054808 -0.702113    -1.0   
            2022-03-18 -0.775873    -7.0 -0.093871 -0.627809    -1.0   

                        ret6_fwd       adv20  cs_rank_adv20  
permaticker date                                             
104612      2022-01-10         1   184505.90         2575.0  
            2022-01-11         0   191152.90         2625.0  
            2022-01-12         0   196837.85         2662.0  
            2022-01-13         0   199983.05         2713.0  
            2022-01-14         0   206573.50         2763.0  
...                          ...         ...            ...  
637794      2022-03-14         0  3890000.10         5470.0  
            2022-03-15         0  4254923.55         5507.0  
            2022-03-16         0  4176195.25         5494.0  
            2022-03-17         0  4162609.00         5483.0  
            2022-03-18         0  4158389.85         5462.0  

[5267783 rows x 20 columns]

In [51]:
eng_pre_small = eng_pre.head(500000)

In [52]:
feat = [f'alpha{i+1}' for i in range(6)]
target_col = 'ret6_fwd'
X = eng_pre_small[feat]
y = eng_pre_small[target_col]

In [54]:
# s_time_chunk = time.time()
# clf = RandomForestClassifier(n_estimators=100,class_weight='balanced_subsample',criterion='entropy')
# scores = cross_validate(clf, X, y, cv=5, scoring=('accuracy', 'f1', 'precision', 'recall'),return_train_score=True)
# e_time_chunk = time.time()
# print(f"Total time: {e_time_chunk - s_time_chunk} sec")

s_time_chunk = time.time()
clf = RandomForestClassifier(n_estimators=100,class_weight='balanced_subsample',criterion='entropy', n_jobs=-1)
scores = cross_validate(clf, X, y, cv=5, scoring=('accuracy', 'f1', 'precision', 'recall'),return_train_score=True)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

KeyboardInterrupt: 

In [47]:
X

alpha1    alpha2    alpha3  alpha4    alpha5  \
permaticker date                                                         
104612      2022-01-10 -0.236569 -0.371428 -0.963897    -5.0 -0.119659   
            2022-01-11 -0.111537  0.306276 -0.959182    -5.0 -0.523857   
            2022-01-12 -0.058054  0.500763 -0.964285    -6.0 -0.640489   
            2022-01-13 -0.346554  0.367854 -0.936529    -6.0 -0.340535   
            2022-01-14 -0.277474 -0.389261 -0.930440    -6.0 -0.312737   
...                          ...       ...       ...     ...       ...   
637794      2022-03-14 -0.166695  0.637373 -0.922962    -7.0 -0.777463   
            2022-03-15 -0.102859  0.430296 -0.919751    -7.0 -0.081093   
            2022-03-16 -0.368640  0.371101 -0.904994    -7.0 -0.493559   
            2022-03-17 -0.171894 -0.426732 -0.879794    -6.0 -0.054808   
            2022-03-18 -0.111319 -0.419262 -0.775873    -7.0 -0.093871   

                          alpha6  
permaticker date                  
104612      2022-01-10 -0.896131  
            2022-01-11 -0.884942  
            2022-01-12 -0.859963  
            2022-01-13 -0.819221  
            2022-01-14 -0.843895  
...                          ...  
637794      2022-03-14 -0.818239  
            2022-03-15 -0.804018  
            2022-03-16 -0.742509  
            2022-03-17 -0.702113  
            2022-03-18 -0.627809  

[5267783 rows x 6 columns]